In [1]:
import pandas as pd
import numpy as np
import os
import shutil

In [2]:
folder_path = '/ihome/alee/jis117/MB/prep_all'  # Replace with the actual path to your folder
file_prefix = 'All'
file_extension = '.csv'

X_file_names = []
Y_file_names = []

# Iterate over files in the folder
for filename in os.listdir(folder_path):
    if filename.endswith(file_extension) and file_prefix in filename and 'knockoff' not in filename:
        if filename.startswith('Allx'):
            X_file_names.append(filename)
        elif filename.startswith('Ally'):
            Y_file_names.append(filename)

# Sort the file names based on the numerical part
X_file_names.sort(key=lambda x: int(x.split("_")[1].split('.')[0]))
Y_file_names.sort(key=lambda x: int(x.split("_")[1].split('.')[0]))

In [4]:
selected_associations = [];
# Print the file names in order
for i in range(138):
    X_file_name = X_file_names[i];
###########################################
#Generate knockoff data using Cholesky_LU.
    from DL.knockoff.KnockoffGenerator import KnockoffGenerator;
    generator = KnockoffGenerator();
    knockoff_file_path = generator.Chol_Lu_knockoff(folder_path, X_file_name);
    X_knockoff_data = pd.read_csv(knockoff_file_path);

#############################################
# After generating the knockoff data, run DNN
    Y_file_name = Y_file_names[i];
    
#nutrient_data
    original_data_Y = pd.read_csv(folder_path+os.path.sep+Y_file_name);
    original_data_X = pd.read_csv(folder_path+os.path.sep+X_file_name);
    
# original_data_Y

    X_values = X_knockoff_data.values;
    Y_values = original_data_Y.values;
    
    p_num = int(X_values.shape[1] / 2);
    n = X_values.shape[0];
    
    X_origin = X_values[:, 0:p_num];
    X_knockoff = X_values[:, p_num:];

    x3D_train = np.zeros((n, p_num, 2));
    x3D_train[:, :, 0] = X_origin;
    x3D_train[:, :, 1] = X_knockoff;
    label_train = Y_values;
    
    coeff = 0.05 * np.sqrt(2.0 * np.log(p_num) / n);

    n_outputs = original_data_Y.shape[1];

#Save the DNN output to the following directory.
    result_dir = 'data/FMT/DNN_result/';
    if not os.path.exists(result_dir):
        os.makedirs(result_dir);
    
    from DL.DNN.DNN import DNN;
    dnn = DNN();
    model = dnn.build_DNN(p_num, n_outputs, coeff);
    callback = DNN.Job_finish_Callback(result_dir,p_num);
    dnn.train_DNN(model, x3D_train, label_train,callback);

#######################################################
#Apply FDR control to DNN result
    from DL.FDR.FDR_control import FDR_control;
    control = FDR_control();
    selected_features = control.controlFilter(folder_path +os.path.sep+ X_file_name, "/ihome/alee/jis117/causalDeepVASE/data/FMT/DNN_result", offset=1, q=0.001);
#print(selected_features)
##########################################################    
    shutil. rmtree(result_dir)
#Save the selected associations
    for ele in selected_features:
        feature1 = ele[0];
        feature2 = original_data_Y.columns[0];
        w = ele[1];
        selected_associations.append({"Feature1":feature1,"Feature2":feature2,"W":w});
pd.DataFrame(selected_associations).to_csv("data/FMT/DNN_selected_all_0.001.csv")

__init__
(137, 137)
(186, 137)
['K1', 'K2', 'K3', 'K4', 'K5', 'K6', 'K7', 'K8', 'K9', 'K10', 'K11', 'K12', 'K13', 'K14', 'K15', 'K16', 'K17', 'K18', 'K19', 'K20', 'K21', 'K22', 'K23', 'K24', 'K25', 'K26', 'K27', 'K28', 'K29', 'K30', 'K31', 'K32', 'K33', 'K34', 'K35', 'K36', 'K37', 'K38', 'K39', 'K40', 'K41', 'K42', 'K43', 'K44', 'K45', 'K46', 'K47', 'K48', 'K49', 'K50', 'K51', 'K52', 'K53', 'K54', 'K55', 'K56', 'K57', 'K58', 'K59', 'K60', 'K61', 'K62', 'K63', 'K64', 'K65', 'K66', 'K67', 'K68', 'K69', 'K70', 'K71', 'K72', 'K73', 'K74', 'K75', 'K76', 'K77', 'K78', 'K79', 'K80', 'K81', 'K82', 'K83', 'K84', 'K85', 'K86', 'K87', 'K88', 'K89', 'K90', 'K91', 'K92', 'K93', 'K94', 'K95', 'K96', 'K97', 'K98', 'K99', 'K100', 'K101', 'K102', 'K103', 'K104', 'K105', 'K106', 'K107', 'K108', 'K109', 'K110', 'K111', 'K112', 'K113', 'K114', 'K115', 'K116', 'K117', 'K118', 'K119', 'K120', 'K121', 'K122', 'K123', 'K124', 'K125', 'K126', 'K127', 'K128', 'K129', 'K130', 'K131', 'K132', 'K133', 'K134', 'K13

Epoch 1/20
186/186 [==============================] - 2s 9ms/step - loss: 271.5986
on_epoch_end
h_local1_weight = (137, 2, 1)
h_local2_weight = (137, 1, 1)
h0 = (137, 2)
h0_abs = (137, 2)
h1 = (137, 137)
h2 = (137, 137)
h3 = (137, 1)
W1 = (137, 137)
W2 = (137, 137)
W3 = (137, 1)
Epoch 2/20
186/186 [==============================] - 0s 153us/step - loss: 251.5454
on_epoch_end
h_local1_weight = (137, 2, 1)
h_local2_weight = (137, 1, 1)
h0 = (137, 2)
h0_abs = (137, 2)
h1 = (137, 137)
h2 = (137, 137)
h3 = (137, 1)
W1 = (137, 137)
W2 = (137, 137)
W3 = (137, 1)
Epoch 3/20
186/186 [==============================] - 0s 160us/step - loss: 217.6281
on_epoch_end
h_local1_weight = (137, 2, 1)
h_local2_weight = (137, 1, 1)
h0 = (137, 2)
h0_abs = (137, 2)
h1 = (137, 137)
h2 = (137, 137)
h3 = (137, 1)
W1 = (137, 137)
W2 = (137, 137)
W3 = (137, 1)
Epoch 4/20
186/186 [==============================] - 0s 165us/step - loss: 157.5548
on_epoch_end
h_local1_weight = (137, 2, 1)
h_local2_weight = (137, 1, 1

on_epoch_end
h_local1_weight = (137, 2, 1)
h_local2_weight = (137, 1, 1)
h0 = (137, 2)
h0_abs = (137, 2)
h1 = (137, 137)
h2 = (137, 137)
h3 = (137, 1)
W1 = (137, 137)
W2 = (137, 137)
W3 = (137, 1)
Epoch 6/20
186/186 [==============================] - 0s 148us/step - loss: 39.8324
on_epoch_end
h_local1_weight = (137, 2, 1)
h_local2_weight = (137, 1, 1)
h0 = (137, 2)
h0_abs = (137, 2)
h1 = (137, 137)
h2 = (137, 137)
h3 = (137, 1)
W1 = (137, 137)
W2 = (137, 137)
W3 = (137, 1)
Epoch 7/20
186/186 [==============================] - 0s 158us/step - loss: 27.1079
on_epoch_end
h_local1_weight = (137, 2, 1)
h_local2_weight = (137, 1, 1)
h0 = (137, 2)
h0_abs = (137, 2)
h1 = (137, 137)
h2 = (137, 137)
h3 = (137, 1)
W1 = (137, 137)
W2 = (137, 137)
W3 = (137, 1)
Epoch 8/20
186/186 [==============================] - 0s 153us/step - loss: 28.1490
on_epoch_end
h_local1_weight = (137, 2, 1)
h_local2_weight = (137, 1, 1)
h0 = (137, 2)
h0_abs = (137, 2)
h1 = (137, 137)
h2 = (137, 137)
h3 = (137, 1)
W1 = (

Epoch 10/20
186/186 [==============================] - 0s 153us/step - loss: 21.1299
on_epoch_end
h_local1_weight = (137, 2, 1)
h_local2_weight = (137, 1, 1)
h0 = (137, 2)
h0_abs = (137, 2)
h1 = (137, 137)
h2 = (137, 137)
h3 = (137, 1)
W1 = (137, 137)
W2 = (137, 137)
W3 = (137, 1)
Epoch 11/20
186/186 [==============================] - 0s 160us/step - loss: 20.2648
on_epoch_end
h_local1_weight = (137, 2, 1)
h_local2_weight = (137, 1, 1)
h0 = (137, 2)
h0_abs = (137, 2)
h1 = (137, 137)
h2 = (137, 137)
h3 = (137, 1)
W1 = (137, 137)
W2 = (137, 137)
W3 = (137, 1)
Epoch 12/20
186/186 [==============================] - 0s 156us/step - loss: 19.1044
on_epoch_end
h_local1_weight = (137, 2, 1)
h_local2_weight = (137, 1, 1)
h0 = (137, 2)
h0_abs = (137, 2)
h1 = (137, 137)
h2 = (137, 137)
h3 = (137, 1)
W1 = (137, 137)
W2 = (137, 137)
W3 = (137, 1)
Epoch 13/20
186/186 [==============================] - 0s 156us/step - loss: 18.3687
on_epoch_end
h_local1_weight = (137, 2, 1)
h_local2_weight = (137, 1,

on_epoch_end
h_local1_weight = (137, 2, 1)
h_local2_weight = (137, 1, 1)
h0 = (137, 2)
h0_abs = (137, 2)
h1 = (137, 137)
h2 = (137, 137)
h3 = (137, 1)
W1 = (137, 137)
W2 = (137, 137)
W3 = (137, 1)
Epoch 15/20
186/186 [==============================] - 0s 169us/step - loss: 17.6444
on_epoch_end
h_local1_weight = (137, 2, 1)
h_local2_weight = (137, 1, 1)
h0 = (137, 2)
h0_abs = (137, 2)
h1 = (137, 137)
h2 = (137, 137)
h3 = (137, 1)
W1 = (137, 137)
W2 = (137, 137)
W3 = (137, 1)
Epoch 16/20
186/186 [==============================] - 0s 155us/step - loss: 16.9940
on_epoch_end
h_local1_weight = (137, 2, 1)
h_local2_weight = (137, 1, 1)
h0 = (137, 2)
h0_abs = (137, 2)
h1 = (137, 137)
h2 = (137, 137)
h3 = (137, 1)
W1 = (137, 137)
W2 = (137, 137)
W3 = (137, 1)
Epoch 17/20
186/186 [==============================] - 0s 150us/step - loss: 16.3809
on_epoch_end
h_local1_weight = (137, 2, 1)
h_local2_weight = (137, 1, 1)
h0 = (137, 2)
h0_abs = (137, 2)
h1 = (137, 137)
h2 = (137, 137)
h3 = (137, 1)
W1 

...

__init__parameters
[layer]: Input	[shape]: [None, 137, 2] 

[layer]: LocallyConnected1D	[shape]: [None, 137, 1] 

[layer]: LocallyConnected1D	[shape]: [None, 137, 1] 

[layer]: Flatten	[shape]: [None, None] 

[layer]: Dense	[shape]: [None, 137] 

[layer]: Dense	[shape]: [None, 137] 

[layer]: Dense	[shape]: [None, 1] 

Epoch 1/20
186/186 [==============================] - 1s 7ms/step - loss: 53.5678
on_epoch_end
h_local1_weight = (137, 2, 1)
h_local2_weight = (137, 1, 1)
h0 = (137, 2)
h0_abs = (137, 2)
h1 = (137, 137)
h2 = (137, 137)
h3 = (137, 1)
W1 = (137, 137)
W2 = (137, 137)
W3 = (137, 1)
Epoch 2/20
186/186 [==============================] - 0s 154us/step - loss: 45.1084
on_epoch_end
h_local1_weight = (137, 2, 1)
h_local2_weight = (137, 1, 1)
h0 = (137, 2)
h0_abs = (137, 2)
h1 = (137, 137)
h2 = (137, 137)
h3 = (137, 1)
W1 = (137, 137)
W2 = (137, 137)
W3 = (137, 1)
Epoch 3/20
186/186 [==============================] - 0s 154us/step - loss: 35.2081
on_epoch_end
h_local1_weight = (137

186/186 [==============================] - 0s 124us/step - loss: 27.1892
on_epoch_end
h_local1_weight = (137, 2, 1)
h_local2_weight = (137, 1, 1)
h0 = (137, 2)
h0_abs = (137, 2)
h1 = (137, 137)
h2 = (137, 137)
h3 = (137, 1)
W1 = (137, 137)
W2 = (137, 137)
W3 = (137, 1)
Epoch 5/20
186/186 [==============================] - 0s 121us/step - loss: 24.8318
on_epoch_end
h_local1_weight = (137, 2, 1)
h_local2_weight = (137, 1, 1)
h0 = (137, 2)
h0_abs = (137, 2)
h1 = (137, 137)
h2 = (137, 137)
h3 = (137, 1)
W1 = (137, 137)
W2 = (137, 137)
W3 = (137, 1)
Epoch 6/20
186/186 [==============================] - 0s 118us/step - loss: 22.6479
on_epoch_end
h_local1_weight = (137, 2, 1)
h_local2_weight = (137, 1, 1)
h0 = (137, 2)
h0_abs = (137, 2)
h1 = (137, 137)
h2 = (137, 137)
h3 = (137, 1)
W1 = (137, 137)
W2 = (137, 137)
W3 = (137, 1)
Epoch 7/20
186/186 [==============================] - 0s 121us/step - loss: 20.7221
on_epoch_end
h_local1_weight = (137, 2, 1)
h_local2_weight = (137, 1, 1)
h0 = (137, 

Epoch 9/20
186/186 [==============================] - 0s 157us/step - loss: 17.3682
on_epoch_end
h_local1_weight = (137, 2, 1)
h_local2_weight = (137, 1, 1)
h0 = (137, 2)
h0_abs = (137, 2)
h1 = (137, 137)
h2 = (137, 137)
h3 = (137, 1)
W1 = (137, 137)
W2 = (137, 137)
W3 = (137, 1)
Epoch 10/20
186/186 [==============================] - 0s 153us/step - loss: 15.9902
on_epoch_end
h_local1_weight = (137, 2, 1)
h_local2_weight = (137, 1, 1)
h0 = (137, 2)
h0_abs = (137, 2)
h1 = (137, 137)
h2 = (137, 137)
h3 = (137, 1)
W1 = (137, 137)
W2 = (137, 137)
W3 = (137, 1)
Epoch 11/20
186/186 [==============================] - 0s 157us/step - loss: 14.7410
on_epoch_end
h_local1_weight = (137, 2, 1)
h_local2_weight = (137, 1, 1)
h0 = (137, 2)
h0_abs = (137, 2)
h1 = (137, 137)
h2 = (137, 137)
h3 = (137, 1)
W1 = (137, 137)
W2 = (137, 137)
W3 = (137, 1)
Epoch 12/20
186/186 [==============================] - 0s 158us/step - loss: 13.5397
on_epoch_end
h_local1_weight = (137, 2, 1)
h_local2_weight = (137, 1, 

186/186 [==============================] - 0s 163us/step - loss: 14.0544
on_epoch_end
h_local1_weight = (137, 2, 1)
h_local2_weight = (137, 1, 1)
h0 = (137, 2)
h0_abs = (137, 2)
h1 = (137, 137)
h2 = (137, 137)
h3 = (137, 1)
W1 = (137, 137)
W2 = (137, 137)
W3 = (137, 1)
Epoch 15/20
186/186 [==============================] - 0s 159us/step - loss: 13.1688
on_epoch_end
h_local1_weight = (137, 2, 1)
h_local2_weight = (137, 1, 1)
h0 = (137, 2)
h0_abs = (137, 2)
h1 = (137, 137)
h2 = (137, 137)
h3 = (137, 1)
W1 = (137, 137)
W2 = (137, 137)
W3 = (137, 1)
Epoch 16/20
186/186 [==============================] - 0s 152us/step - loss: 12.3488
on_epoch_end
h_local1_weight = (137, 2, 1)
h_local2_weight = (137, 1, 1)
h0 = (137, 2)
h0_abs = (137, 2)
h1 = (137, 137)
h2 = (137, 137)
h3 = (137, 1)
W1 = (137, 137)
W2 = (137, 137)
W3 = (137, 1)
Epoch 17/20
186/186 [==============================] - 0s 151us/step - loss: 11.6324
on_epoch_end
h_local1_weight = (137, 2, 1)
h_local2_weight = (137, 1, 1)
h0 = (13

186/186 [==============================] - 0s 114us/step - loss: 10.3753
on_epoch_end
h_local1_weight = (137, 2, 1)
h_local2_weight = (137, 1, 1)
h0 = (137, 2)
h0_abs = (137, 2)
h1 = (137, 137)
h2 = (137, 137)
h3 = (137, 1)
W1 = (137, 137)
W2 = (137, 137)
W3 = (137, 1)
Epoch 20/20
186/186 [==============================] - 0s 135us/step - loss: 9.8664
on_epoch_end
h_local1_weight = (137, 2, 1)
h_local2_weight = (137, 1, 1)
h0 = (137, 2)
h0_abs = (137, 2)
h1 = (137, 137)
h2 = (137, 137)
h3 = (137, 1)
W1 = (137, 137)
W2 = (137, 137)
W3 = (137, 1)
__init__
274
